# Homework for module one dock terraform

In [44]:
import pandas as pd
import pathlib
import requests

## Download required files

In [45]:
# Download the November 2025 green taxi Parquet file
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet"
local_path = pathlib.Path("green_tripdata_2025-11.parquet")

if not local_path.exists():
    with requests.get(url, stream=True, timeout=30) as r:
        r.raise_for_status()
        with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

print(f"File available at: {local_path.resolve()}")

File available at: E:\myprojects\data-engineering-zoomcamp\01-docker-terraform\01-homework\green_tripdata_2025-11.parquet


In [46]:
# Download taxi zone lookup CSV file
zone_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
zone_path = pathlib.Path("taxi_zone_lookup.csv")

if not zone_path.exists():
    with requests.get(zone_url, stream=True, timeout=30) as r:
        r.raise_for_status()
        with open(zone_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

print(f"Lookup CSV available at: {zone_path.resolve()}")

Lookup CSV available at: E:\myprojects\data-engineering-zoomcamp\01-docker-terraform\01-homework\taxi_zone_lookup.csv


## Read the files

### Green Taxi data in Nov 2025

In [47]:
df_green = pd.read_parquet(
    "green_tripdata_2025-11.parquet",
    engine="pyarrow",
)

In [48]:
df_green.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


### Look up table

In [49]:
df_lookup = pd.read_csv('taxi_zone_lookup.csv')

In [50]:
df_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


## Answer for question 3-6

### Question 3 Counting short trips

In [51]:
# count how many trips has less than 1 mile distance
short_trips = df_green[(df_green['trip_distance'] <= 1.0) & (df_green['lpep_pickup_datetime'].dt.date < pd.to_datetime('2025-12-01').date())]
print(f"Number of trips with less than 1 mile distance: {len(short_trips)}")

Number of trips with less than 1 mile distance: 8007


###  Question 4 Longest trip for each day

In [52]:
# find date with maximum trip distance
max_trip_distance = df_green[df_green['trip_distance'] < 100].sort_values(by='trip_distance', ascending=False).head(1)
max_distance = max_trip_distance['trip_distance'].iloc[0]
pickup_datetime = max_trip_distance['lpep_pickup_datetime'].iloc[0]
print(f"Maximum trip distance: {max_distance} miles")
print(f"Date with maximum trip distance: {pickup_datetime.date()}")

Maximum trip distance: 88.03 miles
Date with maximum trip distance: 2025-11-14


### Question 5 Biggest pickup zone

In [53]:
df_2019_10_18 = df_green[df_green['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()]
pickup_totals = df_2019_10_18.groupby('PULocationID')['total_amount'].sum().sort_values(ascending=False)
top_pickup_location = pickup_totals.index[0]
print(f"Top pickup locations in total_amount for 2025-11-18 is {top_pickup_location}")

Top pickup locations in total_amount for 2025-11-18 is 74


In [ ]:
print(df_lookup[df_lookup['LocationID'] == top_pickup_location])

    LocationID    Borough               Zone service_zone
73          74  Manhattan  East Harlem North    Boro Zone


### Question 6 Largest tip

In [58]:
selected_locations = df_green[df_green['PULocationID'] == top_pickup_location]
largest_tip = selected_locations[selected_locations['tip_amount'] == selected_locations['tip_amount'].max()]
largest_tip
drop_off_location_id = largest_tip['DOLocationID'].values[0]
print(f"The drop-off location ID for the trip with the largest tip from PULocationID 74 is: {drop_off_location_id}")

The drop-off location ID for the trip with the largest tip from PULocationID 74 is: 263


In [59]:
select_dropoff = df_lookup[df_lookup['LocationID'] == drop_off_location_id]
select_dropoff

,LocationID,Borough,Zone,service_zone
262,263,Manhattan,Yorkville West,Yellow Zone
